#### Importing

In [25]:
import pandas as pd
import numpy as np
from tensorflow.keras        import Sequential, layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing   import MinMaxScaler

### Data Processing

In [4]:
df = pd.read_csv('../data/data_combined_v1.03.csv', encoding='utf-8-sig')
df.columns

Index(['place_id', 'tripadvisor_id', 'fsq_id', 'name', 'business_status',
       'types', 'price_level', 'ta_price_level', 'price_level_combined',
       'rating', 'user_ratings_total', 'ta_rating', 'ta_user_ratings_total',
       'combined_rating', 'geometry.location.lat', 'geometry.location.lng',
       'formatted_address', 'zip_code', 'city', 'international_phone_number',
       'website', 'google_category', 'ta_4sq_cuisine1', 'ta_4sq_cuisine2',
       'ta_4sq_cuisine3', 'ta_4sq_cuisine4', 'ta_4sq_cuisine5',
       'main_category_1', 'main_category_2', 'main_category_3',
       'main_category_4', 'combined_main_category', 'sub_category',
       'neighboorhood_area_km2', 'resident_population_total',
       'population_age_0_14', 'population_15_19', 'population_20_24',
       'population_25_34', 'population_35_44', 'population_45_54',
       'population_54_64', 'population_65+', 'resident_per_km2',
       'brut_income_annual_per_person', 'district', 'district_cluster'],
      dtype='o

In [5]:
df[['google_category']].value_counts()

google_category     
Restaurant              758
Italian restaurant       84
Thai restaurant          34
Fast food restaurant     28
Cafe                     28
                       ... 
Food court                1
Fondue restaurant         1
Fish store                1
Lebanese                  1
0                         1
Length: 143, dtype: int64

In [6]:
df.isna().sum()

place_id                            0
tripadvisor_id                    624
fsq_id                            909
name                                0
business_status                     0
types                               0
price_level                       995
ta_price_level                    785
price_level_combined              194
rating                            219
user_ratings_total                219
ta_rating                         685
ta_user_ratings_total             685
combined_rating                   196
geometry.location.lat               0
geometry.location.lng               0
formatted_address                   0
zip_code                            0
city                                0
international_phone_number        214
website                           934
google_category                     0
ta_4sq_cuisine1                     5
ta_4sq_cuisine2                    65
ta_4sq_cuisine3                   155
ta_4sq_cuisine4                   193
ta_4sq_cuisi

In [7]:
feat = [
    'price_level_combined','geometry.location.lat',
    'geometry.location.lng',#'google_category',
    'neighboorhood_area_km2', 'resident_population_total',
    'population_age_0_14', 'population_15_19', 'population_20_24',
    'population_25_34', 'population_35_44', 'population_45_54',
    'population_54_64', 'population_65+', 'resident_per_km2',
    'brut_income_annual_per_person'
]

targ = ['combined_rating']

In [10]:
df1 = df[feat+targ].copy()

In [14]:
df2 = df1.loc[df1['combined_rating'] > 0]

In [23]:
X = df2[feat]
y = df2[targ]

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [26]:
scaler = MinMaxScaler()

In [29]:
X_scaled = scaler.fit_transform(X_train)

### Neural Network

In [58]:
model = Sequential()
model.add(layers.Dense(15, activation='relu', input_dim=15))
model.add(layers.Dense(3, activation='relu'))
model.add(layers.Dense(3, activation='relu'))
model.add(layers.Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='adam')
#model.compile(loss='binary_crossentropy',optimizer='adam',metrics = 'accuracy')

In [59]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 15)                240       
                                                                 
 dense_37 (Dense)            (None, 3)                 48        
                                                                 
 dense_38 (Dense)            (None, 3)                 12        
                                                                 
 dense_39 (Dense)            (None, 1)                 4         
                                                                 
Total params: 304
Trainable params: 304
Non-trainable params: 0
_________________________________________________________________


In [60]:
model.fit(X_scaled,y_train,batch_size=32, epochs=100)

Epoch 1/100
33/33 [==============================] - 0s 1ms/step - loss: 16.2558
Epoch 2/100
33/33 [==============================] - 0s 1ms/step - loss: 11.8849
Epoch 3/100
33/33 [==============================] - 0s 1ms/step - loss: 5.2848
Epoch 4/100
33/33 [==============================] - 0s 1ms/step - loss: 1.7302
Epoch 5/100
33/33 [==============================] - 0s 1ms/step - loss: 1.2909
Epoch 6/100
33/33 [==============================] - 0s 1ms/step - loss: 1.0005
Epoch 7/100
33/33 [==============================] - 0s 1ms/step - loss: 0.7547
Epoch 8/100
33/33 [==============================] - 0s 1ms/step - loss: 0.5667
Epoch 9/100
33/33 [==============================] - 0s 1ms/step - loss: 0.4542
Epoch 10/100
33/33 [==============================] - 0s 1ms/step - loss: 0.3947
Epoch 11/100
33/33 [==============================] - 0s 1ms/step - loss: 0.3744
Epoch 12/100
33/33 [==============================] - 0s 1ms/step - loss: 0.3584
Epoch 13/100
33/33 [===============

In [61]:
model.evaluate(scaler.transform(X_test), y_test)

11/11 [==============================] - 0s 1ms/step - loss: 0.1976


0.19763094186782837

In [71]:
pd.DataFrame(model.predict(scaler.transform(X_test)))

11/11 [==============================] - 0s 768us/step


,0
0,4.157506
1,4.178754
2,4.588519
3,4.343808
4,4.421766
...,...
339,4.438890
340,4.496354
341,4.273372
342,4.368743


In [73]:
y_test.merge(pd.DataFrame(model.predict(scaler.transform(X_test))), axis=1)

11/11 [==============================] - 0s 1ms/step


TypeError: DataFrame.merge() got an unexpected keyword argument 'axis'